In [1]:
import numpy as np
import pandas as pd
import biom
import qiime2 as q2
import q2_sample_classifier
import pickle
from qiime2 import Artifact
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.externals import joblib
from scipy.sparse import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [8]:
!pwd

/Users/huangshi/MyProjects/CMI-IBM/age-prediction/MicrobiomeAgePrediction


## Input train data

In [2]:
train_datafile='/Users/huangshi/MyProjects/CMI-IBM/age-prediction/Input/gut_data/gut_4434.biom'
train_sample_metadata='/Users/huangshi/MyProjects/CMI-IBM/age-prediction/Input/gut_data/gut_4434_map.txt'
train_feature_metadata=''
train_target_field='age'
train_prefix='AGP'

## Input test data

In [10]:
test_datafile='CIAO_dataset/100nt/78846_reference-hit.biom' #gut_4575_rare_sp.csv
test_sample_metadata = 'CIAO_dataset/12685_prep_7611_qiime_20190916-125640.txt' #'10283_20191126-092828.txt'
test_feature_metadata='' 
test_prefix='CIAO'
test_target_field = 'agevisit' #

In [11]:
train_table=biom.load_table(train_datafile)
train_metadata=pd.read_csv(train_sample_metadata, sep='\t')

In [12]:
test_table=biom.load_table(test_datafile)
test_metadata=pd.read_csv(test_sample_metadata, sep='\t')

In [13]:
train_df=train_table.to_dataframe(dense=False)

In [14]:
test_df=test_table.to_dataframe(dense=False)

In [15]:
train_X=train_df.T
train_X.shape

(4434, 65694)

In [16]:
test_X=test_df.T
test_X.shape

(152, 4151)

In [17]:
train_y=train_metadata[train_target_field]

In [18]:
test_y=test_metadata[test_target_field]

In [19]:
pipe = Pipeline([('egr', RandomForestRegressor(max_depth=2, random_state=123, n_estimators=500, n_jobs=4))])
pipe.fit(train_X, train_y)

Pipeline(memory=None,
         steps=[('egr',
                 RandomForestRegressor(bootstrap=True, criterion='mse',
                                       max_depth=2, max_features='auto',
                                       max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=500, n_jobs=4,
                                       oob_score=False, random_state=0,
                                       verbose=0, warm_start=False))],
         verbose=False)

In [21]:
pred_y=pipe.predict(train_X)

In [22]:
R_squared=r2_score(train_y, pred_y)
mse=mean_squared_error(train_y, pred_y)
rmse=np.sqrt(mse)
mae=mean_absolute_error(train_y, pred_y)
print('R-squared: ', R_squared)
print('MSE: ', mse)
print('RMSE: ', rmse)
print('MAE: ', mae)

R-squared:  0.019028518323003674
MSE:  223.78499305980282
RMSE:  14.959444944910317
MAE:  12.650015515968576


In [35]:
pipe.get_params()

{'memory': None,
 'steps': [('egr',
   RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
                         max_features='auto', max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=4,
                         oob_score=False, random_state=0, verbose=0,
                         warm_start=False))],
 'verbose': False,
 'egr': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=4,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=Fals

### Adding preprocessing steps for the test table

### Q: the test data contains 150-nt sequence features while the train data contains 100-nt sequence features.
### Solution: chop the 150-nt sequences into 100 nt ones

In [23]:
def chop_seq_feature_to_nt(x, start=0, end=100):
    '''
    Parameters
    -------
        x: pd.DataFrame 
        A table that contains sequence-like features in the columns
    Return
    -------
        x_dedup: pd.DataFrame
        A table that contain sequence-like features with desired length
    Examples
    -------
    x=pd.DataFrame({'atcttc':[1, 3, 1, 3], 'ttcttc':[1, 3, 3, 1], 
                    'aatttc':[2, 5, 3, 1], 'ttcttc':[2, 5, 3, 1],
                    'aattcc':[2, 5, 3, 1], 'aatatc':[2, 0, 0, 1]})

    '''
    ids=x.columns.tolist()
    new_ids=[i[start:end] for i in ids]
    x.columns=new_ids
    def checkIfDuplicates(listOfElems):
        ''' Check if given list contains any duplicates '''
        if len(listOfElems) == len(set(listOfElems)):
            return False
        else:
            return True
    if(checkIfDuplicates(new_ids)):
        x_dedup=x.sum(axis=1, level=0)
    else:
        x_dedup=x
    return x_dedup

In [ ]:
test_X=chop_seq_feature_to_nt(test_X, start=0, end=100)
test_X.shape

In [25]:
def union_feature_table(train_X, test_X):
    '''
    Parameters
    ----------
    train data : pd.DataFrame
        train data table
    test data : pd.DataFrame
        test data table.
    Returns
    -------
    pd.DataFrame
        A padded test data table
    Examples
    -------
    a=pd.DataFrame({'atc':[1, 3, 1, 3], 'ttc':[1, 3, 3, 1], 'aat':[2, 5, 3, 1], 'ttc':[2, 5, 3, 1]})
    b=pd.DataFrame({'atc':[1, 3, 1, 0], 'tta':[0, 3, 1, 0], 'aaa':[2, 1, 3, 1]})
    feature_padding(a, b)

    '''
    train_feature_ids=train_X.columns.values.tolist()
    test_feature_ids=test_X.columns.values.tolist()
    train_X_uniq_f=list(set(train_feature_ids)-set(test_feature_ids))
    test_X_uniq_f=list(set(test_feature_ids)-set(train_feature_ids))
    test_zero_matrix = np.zeros(shape=(test_X.shape[0], len(train_X_uniq_f)))
    train_zero_matrix = np.zeros(shape=(train_X.shape[0], len(test_X_uniq_f)))
    test_padding_matrix=pd.DataFrame(test_zero_matrix, columns = train_X_uniq_f)
    train_padding_matrix=pd.DataFrame(train_zero_matrix, columns = test_X_uniq_f)

    new_test_X=pd.concat([test_X, test_padding_matrix], axis=1).sort_index(axis=1)
    new_train_X=pd.concat([train_X, train_padding_matrix], axis=1).sort_index(axis=1)
    return new_test_X, new_train_X


In [26]:
a_, b_ = union_feature_table(a, b)
display(a_, b_)

NameError: name 'a' is not defined

In [27]:
def pad_feature(a, b):
    '''
    Parameters
    ----------
    a : pd.DataFrame
        train data table
    b : pd.DataFrame
        test data table.
    Returns
    -------
    pd.DataFrame
        A test table with equal number of
        feature as the train table.
    Examples
    -------
    a=pd.DataFrame({'ttc':[1, 3, 1, 3], 'atc':[1, 3, 3, 1], 'aat':[2, 5, 3, 1], 'tac':[2, 5, 3, 1]})
    b=pd.DataFrame({'atc':[1, 3, 1, 0], 'ttc':[0, 3, 1, 0], 'aaa':[2, 1, 3, 1]})
    pad_feature(a, b)
    
    A	B	C	R
    0	1	0.0	0.0	0.0
    1	3	0.0	0.0	0.0
    2	1	0.0	0.0	0.0
    3	0	0.0	0.0	0.0
    
    '''
    print("The shape of train data: ", a.shape)
    a_feature_ids=a.columns.values.tolist()
    b_feature_ids=b.columns.values.tolist()
    print("The number of features in the original test data: ", len(b_feature_ids))
    a_uniq_f=list(set(a_feature_ids)-set(b_feature_ids))
    ab_shared_f=set(a_feature_ids).intersection(set(b_feature_ids))
    print("The number of features with all zeros in the new test data: ", len(a_uniq_f))
    print("The number of shared features kept in the new test data: ", len(ab_shared_f))
    b_padding_matrix = pd.DataFrame(0, index=b.index, columns=a_uniq_f)
    new_b=pd.concat([b[ab_shared_f], b_padding_matrix], axis=1)
    #print(new_b.shape)
    new_b=new_b[a_feature_ids]
    # only keep feature ids in the train table
    
    print("The shape of new test data: ", new_b.shape)
    return new_b


In [28]:
a=pd.DataFrame({'ttc':[1, 3, 1, 3], 'atc':[1, 3, 3, 1], 'aat':[2, 5, 3, 1], 'tac':[2, 5, 3, 1]})
b=pd.DataFrame({'atc':[1, 3, 1, 0], 'ttc':[0, 3, 1, 0], 'aaa':[2, 1, 3, 1]})
b_=pad_feature(a, b)
b_

The shape of train data:  (4, 4)
The number of features in the original test data:  3
The number of features with all zeros in the new test data:  2
The number of shared features kept in the new test data:  2
The shape of new test data:  (4, 4)


,ttc,atc,aat,tac
0,0,1,0,0
1,3,3,0,0
2,1,1,0,0
3,0,0,0,0


### age prediction on the test dataset after feature padding

In [29]:
train_X.shape

(4434, 65694)

In [30]:
test_X.shape

(152, 4151)

In [31]:
test_X_=pad_feature(train_X, test_X)
#test_X_=pd.SparseDataFrame(test_X_)

The shape of train data:  (4434, 65694)
The number of features in the original test data:  4151
The number of features with all zeros in the new test data:  63359
The number of shared features kept in the new test data:  2335
The shape of new test data:  (152, 65694)


In [32]:
test_X_.shape

(152, 65694)

In [36]:
y_pred = pipe.predict(test_X_)
y_pred 

array([47.53199267, 48.56609068, 47.18598594, 47.6678977 , 47.38379306,
       47.11850387, 47.51542457, 47.18470607, 47.53199267, 48.26575407,
       47.51040513, 47.52785307, 47.53199267, 47.51542457, 47.521985  ,
       47.42821832, 46.90181035, 47.48856014, 46.97941631, 46.62406936,
       47.27374523, 47.53199267, 47.521985  , 47.53199267, 47.59923509,
       47.53199267, 47.53199267, 47.45338377, 47.37243854, 47.56214953,
       47.57030654, 47.17498101, 46.99669383, 46.99386828, 47.49121714,
       47.58454132, 47.51542457, 47.24239077, 47.07751603, 46.798596  ,
       47.5785739 , 47.07460819, 47.34970254, 47.51542457, 47.36079774,
       47.51128497, 47.49383703, 47.06954716, 46.47190978, 47.52949275,
       47.51040513, 47.13520361, 47.53199267, 47.48727968, 47.55663262,
       47.06699676, 47.86891084, 47.51040513, 47.46034604, 47.51542457,
       47.51542457, 47.51542457, 47.51466164, 47.53199267, 47.38788951,
       47.31375606, 47.29058064, 47.51040513, 47.51040513, 48.70

### qiime2

In [506]:
import q2_sample_classifier

In [504]:
! qiime info

System versions
Python version: 3.6.7
QIIME 2 release: 2019.7
QIIME 2 version: 2019.7.0
q2cli version: 2019.7.0

Installed plugins
alignment: 2019.7.0
breakaway: 0+untagged.71.g503723a
composition: 2019.7.0
cutadapt: 2019.7.0
dada2: 2019.7.0
deblur: 2019.7.0
deicode: 0.2.4
demux: 2019.7.0
diversity: 2019.7.0
emperor: 2019.7.0
feature-classifier: 2019.7.0
feature-table: 2019.7.0
fragment-insertion: 2019.7.0
gneiss: 2019.7.0
longitudinal: 2019.7.0
metadata: 2019.7.0
mmvec: 1.0.0
phylogeny: 2019.7.0
quality-control: 2019.7.0
quality-filter: 2019.7.0
qurro: 0.4.0
sample-classifier: 2019.7.1
songbird: 0.9.0
taxa: 2019.7.0
types: 2019.7.0
vsearch: 2019.7.0

Application config directory
/Users/huangshi/anaconda3/envs/qiime2-2019.7/var/q2cli

Getting help
To get help with QIIME 2, visit https://qiime2.org


##  Microbiome age prediction using q2_sample_classifier.classify.predict_regression

In [1040]:
! qiime tools import \
--type 'FeatureTable[Frequency]'\
--input-path /Users/huangshi/MyProjects/CMI-IBM/age-prediction/Input/gut_data/gut_4434.biom \
--output-path /Users/huangshi/MyProjects/CMI-IBM/age-prediction/Input/gut_data/gut_4434.qza

/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/huangshi/

In [91]:
test_metadata_92=test_metadata[test_metadata['agevisit'] != 'not applicable']
ids=test_metadata_92['#SampleID']
test_metadata_92.to_csv('CIAO_dataset/metadata_92.tsv', sep='\t', index=False)

In [81]:
test_X_table_ = biom.Table(test_X_.T.values, test_X_.columns, test_X_.index)
test_X_table_.filter(ids, inplace=True)

65694 x 92 <class 'biom.table.Table'> with 18849 nonzero entries (0% dense)

In [83]:
q2_test_X_table_=q2.Artifact.import_data('FeatureTable[Frequency]', test_X_table_)
q2_test_X_table_.save('CIAO_dataset/100nt/feature-table.92.padding_gut_4434.qza')

'CIAO_dataset/100nt/feature-table.92.padding_gut_4434.qza'

In [45]:
! cd /Users/huangshi/MyProjects/CMI-IBM/age-prediction/MicrobiomeAgePrediction

! ls .

CIAO_dataset
CIAO_predictions
Microbiome Age prediction for new datasets.ipynb
Regressors


In [47]:
! qiime sample-classifier regress-samples \
  --i-table ../Input/gut_data/gut_4434.qza \
  --m-metadata-file ../Input/gut_data/gut_4434_map.txt \
  --m-metadata-column age \
  --p-estimator RandomForestRegressor \
  --p-n-estimators 500 \
  --p-parameter-tuning True \
  --p-n-jobs 8 \
  --p-random-state 123 \
  --output-dir Regressor/16S-100nt-gut_4434/regress-samples

/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/huangshi/

In [48]:
#! qiime sample-classifier fit-regressor \
#--i-table Input/gut_data/gut_4434.qza \
#--m-metadata-file Input/gut_data/gut_4434_map.txt \
#--p-cv 5\
#--m-metadata-column age \
#--p-n-jobs 4 \
#--p-parameter-tuning True \
#--o-sample-estimator Output/gut_4434.regressor.qza \
#--o-feature-importance Output/gut_4434.feature-importance.qza

### predict-regression on the new dataset

In [85]:
! qiime sample-classifier predict-regression \
--i-table CIAO_dataset/100nt/feature-table.92.padding_gut_4434.qza \
--i-sample-estimator  Regressor/16S-100nt-gut_4434/regress-samples/sample_estimator.qza \
--o-predictions CIAO_dataset/16S-100nt-gut_4434-CIAO.92.regressor_prediction.qza

/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/huangshi/

In [86]:
! qiime metadata tabulate \
--m-input-file CIAO_dataset/16S-100nt-gut_4434-CIAO.92.regressor_prediction.qza\
--o-visualization CIAO_dataset/16S-100nt-gut_4434-CIAO.92.regressor_prediction.qzv

/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/huangshi/

In [92]:
!qiime sample-classifier scatterplot \
  --i-predictions CIAO_dataset/16S-100nt-gut_4434-CIAO.92.regressor_prediction.qza \
  --m-truth-file CIAO_dataset/metadata_92.tsv \
  --m-truth-column agevisit \
  --o-visualization CIAO_dataset/16S-100nt-gut_4434-CIAO.92.regressor_prediction.scatter.qzv

/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/huangshi/anaconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/huangshi/